In [ ]:
# torch cash 초기화
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [4]:
import sys
sys.path.append('~/aiffel/aiffelthon')

In [ ]:
!ls

In [1]:
!pip install rouge_score
!pip install datasets==1.0.2
!pip install transformers==4.24.0
!pip install transformer-utils
!pip install packaging

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=eb2e34a72cd4b033f382e03ede960603c03d27cceea8cc67defdd68d82231714
  Stored in directory: /aiffel/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score
     |████████████████████████████████| 1.8 MB 6.8 MB/s            
  Attempting uninstall: datasets
    Found existing installation: datasets 1.14.0
    Uninstalling datasets-1.14.0:
      Successfully uninstalled datasets-1.14.0
     |████████████████████████████████| 5.5 MB 6.2 MB/s            
     |████████████████████████████████| 7.6 MB 77.1 MB/s            
     |████████████████████████████████| 163 kB 96.1 MB/s            
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: huggingface-hub
    Found e

     |████████████████████████████████| 86 kB 10.4 MB/s            


In [3]:
# 필요한 라이브러리 불러오기
import datasets
from datasets import load_dataset
from datasets import Dataset
import transformers
import pandas as pd
import re
import random
import torch

#Tokenizer
from transformers import BertTokenizerFast

#Encoder-Decoder Model
from transformers import EncoderDecoderModel

#Tokenizer, Encoder-Decoder Model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
#Training
from seq2seq_trainer import Seq2SeqTrainer
from transformers import TrainingArguments
from seq2seq_training_args import Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

In [1]:
# 경로 지정
%cd ~/aiffel/aiffelthon/csv

/aiffel/aiffel/aiffelthon/csv


In [2]:
!unzip /aiffel/aiffel/aiffelthon/make_csv_.zip

Archive:  /aiffel/aiffel/aiffelthon/make_csv_.zip
  inflating: val_2-3sent.csv         
  inflating: val_Sum1.csv            
  inflating: val_20per.csv           
  inflating: train_20per.csv         
  inflating: train_2-3sent.csv       
  inflating: train_Sum1.csv          


In [ ]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/legacy/seq2seq/seq2seq_trainer.py

In [ ]:
#!wget https://github.com/huggingface/transformers/blob/main/src/transformers/utils/import_utils.py

In [ ]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/models/encoder_decoder/modeling_encoder_decoder.py

In [ ]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/configuration_utils.py

In [5]:
# 데이터 불러오기
train_df = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/train_Sum1.csv')
#train_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
#train_df.rename(columns = {"input": "Text"}, inplace = True)
#train_df.rename(columns = {"sentence_onesent": "Summary"}, inplace = True)
#train_df.rename(columns = {"sentence_per_20": "Summary"}, inplace = True)

val_df = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/val_Sum1.csv')
#val_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
#val_df.rename(columns = {"input": "Text"}, inplace = True)
#val_df.rename(columns = {"summary_onesent": "Summary"}, inplace = True)
#val_df.rename(columns = {"summary_per_20": "Summary"}, inplace = True)
print(len(train_df), len(val_df))

146771 18300


In [6]:
train_df.head()

,Text,Summary,Category
0,"지어내버린 대목부터는 흥분이 버썩 줄어지었다 ──. ""선생님! 또 기침이 나고 토...",자신을 배반한 제 계집과 세상이 엎드려 죄 사하기를 빌 때까지 죽지 아니하겠다는 H...,literature
1,이 송아지가 젖을 떼우고 집으로 끌고 오던 날은 첨지는 개선장군이 성안 에 들어올...,젖뗀 송아지를 집으로 끌고 오던 날 어깨춤을 추면서 소 들어간다고 고함을 지르는 첨...,literature
2,어떤 여름날 밤 손주딸에게 관한 불길한 꿈을 꾼 이 한머니는 이튿날 조반 후에 생...,손주딸의 관한 불길한 꿈을 꾼 한머니는 백여 리 떨어진 손주딸의 집에를 가보기로 하...,literature
3,이러한 가운데서 왕후는 자기의 입장을 위태롭게 여기고 겸하여 장래 자기 의 몸으로...,자기는 태자의 위를 동경하거나 부러워한 적이 없으며 이 나라의 충성된 신자로서 공주...,literature
4,“가겠소.” “언니가 나오시면 일러드 리겠으니 그때까지는 찾아오지 않으시는 것이 ...,자신의 신변을 염려하여 빠른 걸음으로 골목을 빠져나와 침착한 의식을 회복하면서 어수...,literature


In [15]:
train_df['Category'].value_counts()

speech        32000
minute        27200
news_r        21600
briefing      16000
literature     9600
narration      8371
his_cul        8000
paper          8000
edit           8000
public         8000
Name: Category, dtype: int64

In [4]:
# 10번째 row만 추출
# train_df = train_df.iloc[range(0, len(train_df), 2)]
train_df = train_df.iloc[1::2, :]
#val_df = val_df.iloc[range(0, len(val_df), 2)]
print(len(train_df), len(val_df))

62845 18300


In [5]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (...) 제거
    sentence = re.sub('"','', sentence) # 쌍따옴표 제거
    sentence = re.sub("'",'', sentence) # 따옴표 제거
    sentence = re.sub('\n','', sentence) # \n " 제거
    sentence = re.sub('.{2,3}\W{0,1}기자','', sentence) # 기자 이름 제거
    sentence = re.sub(r'[?.!,][/?.!,]', '', sentence) # 여러개 문장 부호를 하나의 문장부호로 바꿉니다
    sentence = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣a-z0-9]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거

    return sentence

In [6]:
# 전체 Text 데이터에 대한 전처리 (1)
from tqdm import tqdm
clean_text = []

for s in tqdm(train_df['Text']):
    clean_text.append(preprocess_sentence(s))
    
train_df['Text'] = clean_text

100%|██████████| 62845/62845 [00:12<00:00, 4887.65it/s]


In [7]:
# 전체 headlines 데이터에 대한 전처리 
clean_headlines = []

for s in tqdm(train_df['Summary']):
      clean_headlines.append(preprocess_sentence(s))
        
train_df['Summary'] = clean_headlines

100%|██████████| 62845/62845 [00:01<00:00, 42176.16it/s]


In [8]:
# 전체 Text 데이터에 대한 전처리 (1)

clean_text_val = []

for s in tqdm(val_df['Text']):
    clean_text_val.append(preprocess_sentence(s))
    
val_df['Text'] = clean_text_val

100%|██████████| 18300/18300 [00:03<00:00, 5014.73it/s]


In [9]:
# 전체 headlines 데이터에 대한 전처리 
clean_headlines_val = []

for s in tqdm(val_df['Summary']):
      clean_headlines_val.append(preprocess_sentence(s))
        
val_df['Summary'] = clean_headlines_val

100%|██████████| 18300/18300 [00:00<00:00, 43912.86it/s]


In [ ]:
train_df.head()

In [ ]:
val_df.head()

In [10]:
# reset_index 사용
train_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)

In [11]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
#val_len = len(val_df) // 2
val_data = Dataset.from_pandas(val_df.iloc[::2, :])
test_data = Dataset.from_pandas(val_df.iloc[1::2, :])

In [ ]:
print(train_data)
print(val_data)
print(test_data)

In [12]:
# max len setting
encoder_max_length=512
decoder_max_length=128
batch_size = 16

In [13]:
model_checkpoint = 'kykim/bertshared-kor-base'

In [14]:
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/344k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.24k [00:00<?, ?B/s]

In [14]:
tokenizer("전 그저 정신을 잃고 말았습니다.")

{'input_ids': [2, 6019, 17618, 23697, 27790, 4027, 14722, 2016, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#max_input = 512
#max_target = 128
#batch_size = 16
#model_checkpoint = "kykim/bertshared-kor-base"

In [15]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["Summary"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()

    # because RoBERTa automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
    # We have to make sure that the PAD token is ignored
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

In [ ]:
#train_data = train_data.select(range(32))

In [ ]:
#val_data = val_data.select(range(8))

In [ ]:
def preprocess_data(data_to_process):
    #get all the dialogues
    inputs = [dialogue for dialogue in data_to_process['Text']]
    #tokenize the dialogues
    model_inputs = tokenizer(inputs,  max_length=encoder_max_length, padding='max_length', truncation=True)
    #tokenize the summaries
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(data_to_process['Summary'], max_length=decoder_max_length, padding='max_length', truncation=True)

    #set labels
    model_inputs['labels'] = targets['input_ids']
    #return the tokenized data
    #input_ids, attention_mask and labels
    return model_inputs

In [16]:
#processing training data
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"])
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],)

#processing validation data
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"])
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],)

  0%|          | 0/3928 [00:00<?, ?ba/s]

  0%|          | 0/572 [00:00<?, ?ba/s]

In [ ]:
#processing training data
train_data = train_data.map(
    preprocess_data, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"])

train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],)

#processing validation data
val_data = val_data.map(
    preprocess_data, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"])

val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],)

In [ ]:
train_data[0]

In [ ]:
#train_data = train_data.map(preprocess_data, batched = True, remove_columns=["Text", "Summary"])
#val_data = val_data.map(preprocess_data, batched = True, remove_columns=["Text", "Summary"])

In [17]:
# 인코더 모델 불러오기
#from transformers import AutoModelForCausalLM

#bertshared =EncoderDecoderModel.from_pretrained("kykim/bertshared-kor-base")
#bertshared = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model = EncoderDecoderModel.from_pretrained('/aiffel/aiffel/aiffelthon/ch10000/checkpoint-10000/')

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [ ]:
model.save_pretrained(model_checkpoint)

In [ ]:
model.config

In [18]:
# set special tokens
#from transformers import EncoderDecoderConfig
#model.config.decoder_start_token_id = tokenizer.bos_token_id   
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
# set decoding params                               
model.config.max_length = 128
model.config.min_length = 32
model.config.no_repeat_ngram_size = 2
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 2 # 
model.config.vocab_size = model.config.encoder.vocab_size

In [ ]:
@dataclass
class Seq2SeqTrainingArguments(TrainingArguments):
    label_smoothing: Optional[float] = field(
        default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
    )
    sortish_sampler: bool = field(default=False, metadata={"help": "Whether to SortishSamler or not."})
    predict_with_generate: bool = field(
        default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
    )
    adafactor: bool = field(default=False, metadata={"help": "whether to use adafactor"})
    encoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
    )
    decoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
    )
    dropout: Optional[float] = field(default=None, metadata={"help": "Dropout probability. Goes into model.config."})
    attention_dropout: Optional[float] = field(
        default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
    )
    lr_scheduler: Optional[str] = field(
        default="linear", metadata={"help": f"Which lr scheduler to use."}
    )

In [19]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),}

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

### DataCollatorForSeq2Seq의 역할
-  decoder 모델이 있어서 model이라는 인자를 추가하면, pretrained 모델이 `prepare_decoder_input_ids_from_labels`를 가지고 있는 경우에 `decoder_input_ids`를 만든다고 한다.

In [20]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [21]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/aiffel/aiffel/aiffelthon/ch30000/",
    #num_train_epochs=1,  # demo
    max_steps = 30000,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,  # demo
    per_device_eval_batch_size=4,
    learning_rate=3e-05,
    save_steps= 5000,
    eval_steps= 5000,
    warmup_steps=5000,
    #weight_decay=0.1,
    #label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="/aiffel/aiffel/aiffelthon/logs",
    logging_steps=5000,
    save_total_limit=3,
    fp16 = True)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/aiffel/aiffel/aiffelthon/ch/",
    num_train_epochs=1, 
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=3e-05,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_steps=2000,  # set to 2000 for full training
    save_steps=500,  # set to 500 for full training
    eval_steps=500,  # set to 7500 for full training
    warmup_steps=500,  # set to 3000 for full training
    logging_dir="/aiffel/aiffel/aiffelthon/logs",
    save_total_limit=3,
    fp16=True,)
    #max_steps=128)

In [22]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model = model, 
    args = training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [23]:
trainer.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 62845
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 30000
  Number of trainable parameters = 147298320
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to trai

Step,Training Loss
5000,1.394500
10000,1.455100
15000,1.417600
20000,1.161800
25000,1.104200
30000,1.083400


/opt/conda/lib/python3.9/site-packages/transformers/trainer.py:1794: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
Saving model checkpoint to /aiffel/aiffel/aiffelthon/ch30000/checkpoint-5000
Configuration saved in /aiffel/aiffel/aiffelthon/ch30000/checkpoint-5000/config.json
Model weights saved in /aiffel/aiffel/aiffelthon/ch30000/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in /aiffel/aiffel/aiffelthon/ch30000/checkpoint-5000/tokenizer_config.json
Special tokens file saved in /aiffel/aiffel/aiffelthon/ch30000/checkpoint-5000/special_tokens_map.json
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.1



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=30000, training_loss=1.2694332194010416, metrics={'train_runtime': 10019.9797, 'train_samples_per_second': 11.976, 'train_steps_per_second': 2.994, 'total_flos': 4.226262287553331e+16, 'train_loss': 1.2694332194010416, 'epoch': 1.91})

In [25]:
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)
model = EncoderDecoderModel.from_pretrained('/aiffel/aiffel/aiffelthon/ch30000/checkpoint-30000/').to("cuda")
batch_size = 16

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=32, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch
results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["Text"])
pred_str = results["pred"]
label_str = results["Summary"]

loading file vocab.txt from cache at /aiffel/.cache/huggingface/hub/models--kykim--bertshared-kor-base/snapshots/8ea27677d006a547aee2a7753d1ff18a0346860c/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /aiffel/.cache/huggingface/hub/models--kykim--bertshared-kor-base/snapshots/8ea27677d006a547aee2a7753d1ff18a0346860c/tokenizer_config.json
loading configuration file config.json from cache at /aiffel/.cache/huggingface/hub/models--kykim--bertshared-kor-base/snapshots/8ea27677d006a547aee2a7753d1ff18a0346860c/config.json
Model config EncoderDecoderConfig {
  "_commit_hash": "8ea27677d006a547aee2a7753d1ff18a0346860c",
  "_name_or_path": "kykim/bertshared-kor-base",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaske

loading configuration file /aiffel/aiffel/aiffelthon/ch30000/checkpoint-30000/config.json
Model config EncoderDecoderConfig {
  "_commit_hash": null,
  "_name_or_path": "/aiffel/aiffel/aiffelthon/ch10000/checkpoint-10000/",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 2,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": 2,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "embedding_size": 768,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 3,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token

  0%|          | 0/572 [00:00<?, ?ba/s]

In [ ]:
trainer.evaluate()

In [ ]:
print(type(pred_str), type(label_str))

In [ ]:
for num in range(5,4):
    print('predicted sentence : ',pred_str[num])
    print('real sentence : ', label_str[num])
    print('-'*100)

In [23]:
num = 0
print('predicted sentence : ',pred_str[num])
print('\n')
print('real sentence : ', label_str[num])
print('\n')
print('Text', test_data[num])

predicted sentence :  국내에서 처음으로 남의 팔을 이식받은 손 씨의 2020년은 특별한 일이다가 그의 2세가 미국에서 처음으로 남 팔을이식받았다고 했다 손 씨가 미국에서 처음 남의 팔이식을 받은 손 씨 씨 씨의 2세이다


real sentence :  공장에서 일하다 왼쪽 팔을 잃은 그는 팔 이식 수술 후 결혼에 성공하여 아빠가 되는 꿈을 이루었다


Text {'Text': '기다리던 새해가 밝았네요 올해 제가 드디어 아빠가 됩니다 국내에서 처음으로 남의 팔을 이식받은 손진욱 씨의 2020년은 특별하다 지난해부터 기다리던 그의 2세가 오는 8월 태어나기 때문이다 장애인으로 살던 그가 팔 이식 수술 후 결혼에 골인했고 이어 행복한 가정을 꾸려 새해 아빠가 된다 손씨는 2018년 가을 지인의 소개로 만나 교제해 온 여자친구와 지난해 6월 대구에서 부부의 연을 맺었다 이후 운수업 서비스업 등 평범한 30대 직장인으로 생활해왔다 그는 장애가 있던 시절에는 직장 다니고 아이 키우는 평범한 부부를 볼 때 제일 부러웠다며올해는 아빠가 되는 꿈을 이루는 해라고 말했다 손씨는 팔 이식 수술을 받기 위해 병원에 입원한 2017년 1월이 첫 번째 새 인생을 시작한 해라면 지난해 결혼이 두 번째 인생 2020년은 2세가 태어나는 올해는 세 번째 새로운 인생이라고 했다 그는 지난 2015년 공장에서 일하다 사고로 왼쪽 팔을 잃었다 의수를 끼고 생활하다 2017년 2월 대구 w병원과 영남대병원에서 뼈와 신경 근육 혈관 등이 포함된 다른 사람 팔을 이식받았다 새해가 그에게 특별한 이유는 하나 더 있다 수술한 팔이 점점 더 정상인의 팔에 가까워지고 있어서다 희망과 행복이라는 말의 뜻을 이젠 이해하겠다는 손씨는 수술 3년 차를 맞으면서 남의 팔 이 이젠 진짜 내 팔이 된 것 같다 고 했다 그는 수술 초기에는 팔과 손이 저리고 시려지는 고통을 겪었다 컴퓨터나 스마트폰 자판을 두드리는 것 같은 정교한 움직임은 어려웠다 왼쪽 팔이 달렸지만 내 것이 아니라 남의 것이구나 하는

In [24]:
num = 5
print('predicted sentence : ',pred_str[num])
print('\n')
print('real sentence : ', label_str[num])
print('\n')
print('Text', test_data[num])

predicted sentence :  문 대통령과 추 신임 법무장관이 검찰 개혁 의지를 밝혔고 검찰 내부에서는 검찰 인사를 인사 인사를 했다 밝혔다고 문 대통령과 추 법무 장관이 나란히 검찰개혁 의지를 밝혔다


real sentence :  정부 신년회에서 문 대통령과 추 법무장관이 검찰 개혁 의지를 내놓자 검찰 내부에서는 정권에 대한 수사를 압박하려는 의도로 비칠 수 있다는 반응이 나왔다


Text {'Text': '문재인 대통령과 추미애 신임 법무장관이 나란히 검찰 개혁 의지를 밝혔다 윤석열 검찰총장이 참석한 정부 신년회에서다 검찰 내부에서는 검찰 인사를 코앞에 두고 인사권자 두 명이 노골적으로 검찰을 거론한 건 현 정권에 대한 수사를 압박하려는 의도라는 오해를 받을 수 있다 는 반응이 나왔다 윤석열 앞에 두고 검찰이 환부를 못 도려내 2일 오전 문 대통령은 서울 중구 대한상공회의소에서 열린 신년합동인사회에서 권력기관이 국민의 신뢰를 받을 수 있을 때까지 법적 제도적 개혁을 멈추지 않겠다며 국민이 선출한 대통령으로서 헌법에 따라 권한을 다하겠다 고 말했다 권력기관 스스로 개혁에 앞장서 주길 기대한다 고도 했다 이 대통령의 헌법에 따른 권한 이 무엇인지를 두고 의견이 엇갈렸다 대통령이 검찰 경찰 등 권력기관에 대해 행사할 수 있는 헌법상 권한에 인사권이 포함되기 때문이다 문 대통령이 고위공직자범죄수사처나 검찰 개혁에 반대하는 이들에 대해 인사권을 적극 행사하겠다는 뜻을 내비쳤다는 해석이 나온다 검찰 내에서는 다음주 초 고위 간부 인사가 날 거라는 얘기가 파다한 상황이다 이날 추미애 장관은 좀 더 직접적으로 검찰을 겨냥한 발언을 내놨다 추 장관은 검찰의 수사를 의사의 수술에 비유했다 그는 수술칼을 환자에게 여러 번 찔러 병의 원인을 도려내는 것이 명의가 아니라 정확하게 진단하고 정확한 병의 부위를 제대로 도려내는 것이 명의 라고 비판했다 그동안 여권에서 제기해온 검찰의 피의사실 공표나 별건 수사 문제를 지적한 것으로 볼 수 있지만 현재 진행되는 정권 관련

In [ ]:
# step 50000
print("ROUGE 1 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid)
print("ROUGE 2 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid)
print("ROUGE L SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid)

In [ ]:
print("ROUGE 1 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid)
print("ROUGE 2 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid)
print("ROUGE L SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid)